<a href="https://colab.research.google.com/github/ahmedabdulghany7/Competitive-Intelligence-Multi-Agent-System/blob/main/Impact_Scoring_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from enum import Enum
from datetime import datetime, timedelta, timezone
import logging
import re

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# ----- Event & Company enums -----
class EventType(Enum):
    PRODUCT_LAUNCH = "product_launch"
    PRICING_CHANGE = "pricing_change"
    FUNDING = "funding"
    ACQUISITION = "acquisition"
    PARTNERSHIP = "partnership"
    EXPANSION = "expansion"
    LEADERSHIP_CHANGE = "leadership_change"
    CARRIER_DEAL = "carrier_deal"
    FLASH_SALE = "flash_sale"
    BENCHMARK = "benchmark"
    CAMERA_AWARD = "camera_award"
    CERTIFICATION = "certification"
    PREORDER = "preorder"
    OS_UPDATE = "os_update"
    OTHER = "other"

MOBILE_EVENT_SYNONYMS = {
    "launch": EventType.PRODUCT_LAUNCH,
    "product_launch": EventType.PRODUCT_LAUNCH,
    "price_change": EventType.PRICING_CHANGE,
    "pricing": EventType.PRICING_CHANGE,
    "pre-order": EventType.PREORDER,
    "preorder": EventType.PREORDER,
    "carrier": EventType.CARRIER_DEAL,
    "operator": EventType.CARRIER_DEAL,
    "dxomark": EventType.CAMERA_AWARD,
    "benchmark": EventType.BENCHMARK,
    "geekbench": EventType.BENCHMARK,
    "antutu": EventType.BENCHMARK,
    "certification": EventType.CERTIFICATION,
    "flash_sale": EventType.FLASH_SALE,
    "flash": EventType.FLASH_SALE,
    "os_update": EventType.OS_UPDATE,
}

class CompanySize(Enum):
    STARTUP = "startup"      # < $50M rev
    SMALL = "small"          # $50M–$500M
    MEDIUM = "medium"        # $500M–$5B
    LARGE = "large"          # $5B+
    UNKNOWN = "unknown"

@dataclass
class CompetitorProfile:
    name: str
    size: CompanySize
    market_share: Optional[float] = None   # global/regional share (0–1)
    is_direct_competitor: bool = True
    recent_growth: str = "stable"          # "growing" | "stable" | "declining"
    os_ecosystem: str = "android"          # "android" | "ios" | "mixed"
    focus_regions: Optional[List[str]] = None  # e.g., ["KSA","UAE","EG","IN","EU"]

@dataclass
class ImpactScore:
    final_score: float  # 0–10
    competitor_size_score: float
    event_significance_score: float
    timing_score: float
    reasoning: str
    urgency: str  # "immediate" | "high" | "medium" | "low"


In [ ]:
FOCUS_REGIONS_DEFAULT = {"EG","KSA","UAE","QA","KW","OM","BH","IN","EU","US"}
OPERATOR_KEYWORDS = {"vodafone","orange","etisalat","stc","du","mobily","zain"}
REGION_KEYWORDS = {
    "EG":"egypt|cairo|alexandria|eg ",
    "KSA":"ksa|saudi|riyadh|jeddah|dammam",
    "UAE":"uae|dubai|abu dhabi|sharjah",
    "QA":"qatar|doha",
    "KW":"kuwait",
    "OM":"oman|muscat",
    "BH":"bahrain|manama",
    "IN":"india|delhi|mumbai|bangalore",
    "EU":"europe|eu ",
    "US":"united states|usa|us ",
}

def to_aware(dt: datetime) -> datetime:
    if dt is None:
        return datetime.now(timezone.utc)
    return dt if dt.tzinfo else dt.replace(tzinfo=timezone.utc)

def detect_percent_discount(text: str) -> Optional[float]:
    m = re.search(r'(\d{1,2})\s?%', text)
    if m:
        try:
            val = float(m.group(1))
            if 0 < val <= 90:
                return val
        except:
            pass
    return None

def contains_any(text: str, keywords: List[str]) -> bool:
    return any(k in text for k in keywords)

def event_type_from_str(s: str) -> EventType:
    if not s:
        return EventType.OTHER
    s = s.lower()
    if s in MOBILE_EVENT_SYNONYMS:
        return MOBILE_EVENT_SYNONYMS[s]
    try:
        return EventType(s)
    except:
        return EventType.OTHER

def in_focus_regions(text: str, focus: Optional[List[str]]) -> bool:
    if not focus:
        focus = list(FOCUS_REGIONS_DEFAULT)
    t = " " + text.lower() + " "
    for code in focus:
        pat = REGION_KEYWORDS.get(code)
        if pat and re.search(pat, t):
            return True
    return False

def is_flagship_text(text: str) -> bool:
    return contains_any(text, ["ultra","pro max","fold","flip","flagship","top-end"])

def apple_september_boost(ts: datetime, competitor: str, ev: EventType) -> bool:
    return (competitor.lower()=="apple" and ev==EventType.PRODUCT_LAUNCH and ts.month==9)

def samsung_unpacked_boost(ts: datetime, competitor: str, ev: EventType) -> bool:
    return (competitor.lower()=="samsung" and ev==EventType.PRODUCT_LAUNCH and ts.month in (1,2,7))

def mwc_boost(ts: datetime, text: str) -> bool:
    return (ts.month in (2,3) and ("mwc" in text or "barcelona" in text))

def black_white_friday_boost(ts: datetime, text: str) -> bool:
    return (ts.month==11 and ("black friday" in text or "white friday" in text))

def ramadan_mentioned(text: str) -> bool:
    return "ramadan" in text

# ----- Default competitor profiles for mobile -----
def default_mobile_competitors() -> Dict[str, CompetitorProfile]:
    return {
        "Apple":   CompetitorProfile("Apple", CompanySize.LARGE, market_share=0.20, is_direct_competitor=False, recent_growth="growing", os_ecosystem="ios",   focus_regions=["US","EU","KSA","UAE","IN"]),
        "Samsung": CompetitorProfile("Samsung", CompanySize.LARGE, market_share=0.21, is_direct_competitor=True,  recent_growth="growing", os_ecosystem="android",focus_regions=["EU","KSA","UAE","IN","US"]),
        "Xiaomi":  CompetitorProfile("Xiaomi", CompanySize.LARGE, market_share=0.13, is_direct_competitor=True,  recent_growth="growing", os_ecosystem="android",focus_regions=["EU","IN","KSA","UAE","EG"]),
        "OPPO":    CompetitorProfile("OPPO", CompanySize.LARGE, market_share=0.08, is_direct_competitor=True,  recent_growth="stable", os_ecosystem="android",focus_regions=["IN","KSA","UAE","EG"]),
        "vivo":    CompetitorProfile("vivo", CompanySize.LARGE, market_share=0.08, is_direct_competitor=True,  recent_growth="stable", os_ecosystem="android",focus_regions=["IN"]),
        "Huawei":  CompetitorProfile("Huawei", CompanySize.LARGE, market_share=0.07, is_direct_competitor=True,  recent_growth="growing", os_ecosystem="mixed",focus_regions=["KSA","UAE","CN"]),
    }


In [ ]:
class ImpactScoringAgent:
    """
    Impact Scoring Agent – Smartphone Market
    Weights:
      - Competitor size/position (40%)
      - Event significance/type (40%)
      - Timing & context (20%)
    """

    def __init__(self, competitor_profiles: Dict[str, CompetitorProfile], focus_regions: Optional[List[str]]=None):
        self.competitor_profiles = competitor_profiles
        self.focus_regions = focus_regions or list(FOCUS_REGIONS_DEFAULT)

        self.weights = {'competitor_size': 0.4, 'event_significance': 0.4, 'timing': 0.2}

        # Base event scores (mobile-aware)
        self.event_scores = {
            EventType.PRODUCT_LAUNCH: 8.0,
            EventType.PRICING_CHANGE: 7.5,
            EventType.ACQUISITION: 9.0,
            EventType.FUNDING: 6.0,
            EventType.PARTNERSHIP: 6.0,
            EventType.EXPANSION: 7.0,
            EventType.LEADERSHIP_CHANGE: 4.5,
            EventType.CARRIER_DEAL: 7.5,
            EventType.FLASH_SALE: 6.5,
            EventType.BENCHMARK: 5.5,
            EventType.CAMERA_AWARD: 6.0,
            EventType.CERTIFICATION: 5.0,
            EventType.PREORDER: 6.5,
            EventType.OS_UPDATE: 5.5,
            EventType.OTHER: 5.0
        }

        self.size_scores = {
            CompanySize.STARTUP: 4.0,
            CompanySize.SMALL: 5.5,
            CompanySize.MEDIUM: 7.0,
            CompanySize.LARGE: 9.0,
            CompanySize.UNKNOWN: 5.0
        }

    # ---- Public API ----
    def score_signal(self, signal: Dict[str, Any]) -> ImpactScore:
        """
        signal keys: competitor, event_type, text, timestamp (datetime)
        """
        try:
            comp_name = signal.get('competitor', 'Unknown')
            competitor = self.competitor_profiles.get(comp_name)
            if not competitor:
                competitor = self._create_default_profile(comp_name, signal)

            size_score   = self._score_competitor_size(competitor)
            event_score  = self._score_event_significance(signal, competitor)
            timing_score = self._score_timing(signal)

            final_score = (
                size_score * self.weights['competitor_size'] +
                event_score * self.weights['event_significance'] +
                timing_score * self.weights['timing']
            )
            final_score = max(0, min(10, final_score))

            reasoning = self._generate_reasoning(competitor, signal, size_score, event_score, timing_score, final_score)
            urgency   = self._determine_urgency(final_score, timing_score)

            return ImpactScore(
                final_score=round(final_score, 1),
                competitor_size_score=round(size_score, 1),
                event_significance_score=round(event_score, 1),
                timing_score=round(timing_score, 1),
                reasoning=reasoning,
                urgency=urgency
            )
        except Exception as e:
            logger.error(f"Error scoring signal: {e}")
            return self._create_error_score(signal)

    # ---- Components ----
    def _score_competitor_size(self, competitor: CompetitorProfile) -> float:
        base = self.size_scores.get(competitor.size, 5.0)

        # Market share boost
        if competitor.market_share:
            if competitor.market_share > 0.20:
                base += 1.5
            elif competitor.market_share > 0.10:
                base += 1.0
            elif competitor.market_share > 0.05:
                base += 0.5

        # Growth momentum
        if competitor.recent_growth == "growing":
            base += 0.5
        elif competitor.recent_growth == "declining":
            base -= 0.5

        # Direct competitor multiplier (Android vs iOS can still be a threat via halo/aspiration)
        if not competitor.is_direct_competitor:
            base *= 0.8

        return min(10.0, max(0.0, base))

    def _score_event_significance(self, signal: Dict[str, Any], competitor: CompetitorProfile) -> float:
        text = (signal.get('text') or "").lower()
        ev = event_type_from_str(signal.get('event_type', 'other'))

        base = self.event_scores.get(ev, 5.0)

        # High impact keywords
        if contains_any(text, ['revolutionary','breakthrough','first-ever','game-changing']):
            base += 1.5
        elif contains_any(text, ['major','significant','strategic']):
            base += 1.0
        elif contains_any(text, ['new','launches','announces']):
            base += 0.5

        # Price % cuts
        pct = detect_percent_discount(text)
        if ev in (EventType.PRICING_CHANGE, EventType.FLASH_SALE) and pct:
            if pct >= 30:
                base += 1.8
            elif pct >= 15:
                base += 1.2
            else:
                base += 0.6

        # Mobile-specific signals
        if ev == EventType.PRODUCT_LAUNCH:
            if contains_any(text, ['a18','snapdragon','mediatek','exynos','tensor']):
                base += 0.6
            if contains_any(text, ['camera','periscope','ois','hdr','computational photography','dxomark']):
                base += 0.6
            if is_flagship_text(text):
                base += 0.8

        if ev == EventType.CARRIER_DEAL:
            if any(op in text for op in OPERATOR_KEYWORDS):
                base += 0.7
            if in_focus_regions(text, competitor.focus_regions or self.focus_regions):
                base += 0.5

        if ev == EventType.CAMERA_AWARD and 'dxomark' in text:
            base += 0.7

        if ev == EventType.BENCHMARK and contains_any(text, ['antutu','geekbench']):
            base += 0.5

        if ev == EventType.CERTIFICATION and contains_any(text, ['fcc','tenaa','nbtc','imda','bis']):
            base += 0.4

        if ev == EventType.PREORDER and contains_any(text, ['pre-order','preorder','pre order']):
            base += 0.6

        # Region focus boost
        if in_focus_regions(text, competitor.focus_regions or self.focus_regions):
            base += 0.6

        # Big-brand multiplier
        if competitor.size == CompanySize.LARGE:
            base *= 1.15

        return min(10.0, max(0.0, base))

    def _score_timing(self, signal: Dict[str, Any]) -> float:
        ts = to_aware(signal.get('timestamp'))
        now = datetime.now(timezone.utc)
        diff = now - ts

        # Recency
        if diff < timedelta(hours=6):
            score = 10.0
        elif diff < timedelta(days=1):
            score = 8.5
        elif diff < timedelta(days=3):
            score = 7.0
        elif diff < timedelta(weeks=1):
            score = 5.5
        elif diff < timedelta(days=30):
            score = 4.0
        else:
            score = 2.0

        # Business hours (weekday 9–17)
        weekday = ts.weekday()
        hour = ts.hour
        if 9 <= hour <= 17 and weekday < 5:
            score += 0.4
        elif weekday >= 5:
            score -= 0.3

        # Quarter timing
        if ts.month in [3,6,9,12] and ts.day >= 25:
            score += 0.3
        elif ts.month in [1,4,7,10] and ts.day <= 5:
            score += 0.2

        # Mobile calendar/context boosts
        text = (signal.get('text') or "").lower()
        comp = signal.get('competitor','') or ''
        ev = event_type_from_str(signal.get('event_type', 'other'))

        if apple_september_boost(ts, comp, ev):
            score += 0.8
        if samsung_unpacked_boost(ts, comp, ev):
            score += 0.6
        if mwc_boost(ts, text):
            score += 0.5
        if black_white_friday_boost(ts, text) or ramadan_mentioned(text):
            score += 0.4

        return min(10.0, max(1.0, score))

    # ---- Outputs/Reasoning ----
    def _generate_reasoning(self, competitor: CompetitorProfile, signal: Dict[str, Any],
                            size_score: float, event_score: float, timing_score: float, final_score: float) -> str:
        size_desc = {
            CompanySize.STARTUP: "emerging startup",
            CompanySize.SMALL: "small player",
            CompanySize.MEDIUM: "mid-sized OEM",
            CompanySize.LARGE: "top-tier OEM",
            CompanySize.UNKNOWN: "OEM of unknown size"
        }[competitor.size]

        comp_tag = f"{competitor.name} ({size_desc}"
        if competitor.market_share and competitor.market_share > 0.05:
            comp_tag += f", ~{int(competitor.market_share*100)}% share"
        comp_tag += ")"

        ev = event_type_from_str(signal.get('event_type','event'))
        ev_desc = ev.value.replace("_"," ")

        ts = to_aware(signal.get('timestamp'))
        diff = datetime.now(timezone.utc) - ts
        if diff < timedelta(days=1): t_desc = "very recent"
        elif diff < timedelta(weeks=1): t_desc = "recent"
        else: t_desc = "historical"

        assessment = (
            "HIGH IMPACT – act now" if final_score >= 8.0 else
            "MEDIUM-HIGH – monitor closely" if final_score >= 6.0 else
            "MEDIUM – track" if final_score >= 4.0 else
            "LOW – informational"
        )

        return (f"{comp_tag} | Event: {ev_desc} | Timing: {t_desc} | "
                f"Scores — Size:{size_score}/10, Event:{event_score}/10, Timing:{timing_score}/10 | {assessment}")

    def _determine_urgency(self, final_score: float, timing_score: float) -> str:
        if final_score >= 8.0 and timing_score >= 8.5: return "immediate"
        if final_score >= 7.0 or timing_score >= 8.5:  return "high"
        if final_score >= 5.0:                          return "medium"
        return "low"

    # ---- Defaults & Admin ----
    def _create_default_profile(self, competitor_name: str, signal: Dict[str, Any]) -> CompetitorProfile:
        text = (signal.get('text') or "").lower()
        size = CompanySize.UNKNOWN
        if any(k in text for k in ["fortune","global","billion","giant","ecosystem"]):
            size = CompanySize.LARGE
        elif any(k in text for k in ["startup","series a","seed"]):
            size = CompanySize.STARTUP

        os_ecosystem = "android"
        if competitor_name.lower() == "apple": os_ecosystem = "ios"

        prof = CompetitorProfile(
            name=competitor_name, size=size, is_direct_competitor=True,
            os_ecosystem=os_ecosystem, focus_regions=list(self.focus_regions)
        )
        self.add_competitor(prof)
        return prof

    def _create_error_score(self, signal: Dict[str, Any]) -> ImpactScore:
        return ImpactScore(0.0,0.0,0.0,0.0,"Error occurred during scoring analysis","low")

    def add_competitor(self, profile: CompetitorProfile) -> None:
        self.competitor_profiles[profile.name] = profile
        logger.info(f"Added competitor profile: {profile.name}")

    def update_competitor(self, name: str, **updates) -> None:
        if name in self.competitor_profiles:
            c = self.competitor_profiles[name]
            for k,v in updates.items():
                if hasattr(c,k): setattr(c,k,v)
            logger.info(f"Updated competitor: {name}")

In [ ]:
from pprint import pprint

# Setup competitors
competitors = default_mobile_competitors()

# Create scorer
scorer = ImpactScoringAgent(competitors, focus_regions=["KSA","UAE","EG","IN","EU","US"])

now = datetime.now(timezone.utc)
signals = [
    {
        'id': 'A01',
        'competitor': 'Apple',
        'event_type': 'launch',
        'text': 'Apple announces iPhone 16 with A18 chipset efficiency, improved periscope camera; carrier trade-in promos with STC and du in KSA and UAE. Pre-order opens this Friday.',
        'timestamp': now - timedelta(hours=3)
    },
    {
        'id': 'S02',
        'competitor': 'Samsung',
        'event_type': 'pricing_change',
        'text': 'Samsung cuts Galaxy S24 Ultra price by 20% for White Friday in KSA and Egypt; Vodafone/Orange bundles include free buds.',
        'timestamp': now - timedelta(days=2)
    },
    {
        'id': 'X03',
        'competitor': 'Xiaomi',
        'event_type': 'carrier_deal',
        'text': 'Xiaomi signs exclusive operator partnership with Etisalat in UAE for Redmi Note series with aggressive EMI plans.',
        'timestamp': now - timedelta(hours=30)
    },
    {
        'id': 'O04',
        'competitor': 'OPPO',
        'event_type': 'camera_award',
        'text': 'OPPO Find X8 Pro achieves top DxOMark camera score; new computational photography pipeline highlighted.',
        'timestamp': now - timedelta(days=5)
    },
    {
        'id': 'V05',
        'competitor': 'vivo',
        'event_type': 'certification',
        'text': 'vivo V40 Pro appears on NBTC and BIS certification — launch imminent for MENA and India.',
        'timestamp': now - timedelta(days=15)
    },
]

print("=== IMPACT SCORING RESULTS (Mobile) ===\n")
rows = []
for sig in signals:
    score = scorer.score_signal(sig)
    print(f"Signal {sig['id']} | {sig['competitor']} | {sig['event_type']}")
    print(f"Impact: {score.final_score}/10 | Urgency: {score.urgency.upper()}")
    print(f"Breakdown: Size({score.competitor_size_score}) + Event({score.event_significance_score}) + Timing({score.timing_score})")
    print("Reasoning:", score.reasoning)
    print("-"*90)
    rows.append({
        "id": sig["id"],
        "competitor": sig["competitor"],
        "event_type": event_type_from_str(sig.get("event_type")).value,
        "impact": score.final_score,
        "urgency": score.urgency,
        "size": score.competitor_size_score,
        "event": score.event_significance_score,
        "timing": score.timing_score
    })


=== IMPACT SCORING RESULTS (Mobile) ===

Signal A01 | Apple | launch
Impact: 9.4/10 | Urgency: IMMEDIATE
Breakdown: Size(8.4) + Event(10.0) + Timing(10.0)
Reasoning: Apple (top-tier OEM, ~20% share) | Event: product launch | Timing: very recent | Scores — Size:8.4/10, Event:10.0/10, Timing:10.0/10 | HIGH IMPACT – act now
------------------------------------------------------------------------------------------
Signal S02 | Samsung | pricing_change
Impact: 9.5/10 | Urgency: HIGH
Breakdown: Size(10.0) + Event(10.0) + Timing(7.4)
Reasoning: Samsung (top-tier OEM, ~21% share) | Event: pricing change | Timing: recent | Scores — Size:10.0/10, Event:10.0/10, Timing:7.4/10 | HIGH IMPACT – act now
------------------------------------------------------------------------------------------
Signal X03 | Xiaomi | carrier_deal
Impact: 9.4/10 | Urgency: HIGH
Breakdown: Size(10.0) + Event(10.0) + Timing(7.0)
Reasoning: Xiaomi (top-tier OEM, ~13% share) | Event: carrier deal | Timing: recent | Scores — 

In [ ]:
import pandas as pd

df = pd.DataFrame(rows)
df = df[["id","competitor","event_type","impact","urgency","size","event","timing"]].sort_values(by="impact", ascending=False)
df.reset_index(drop=True, inplace=True)
df

,id,competitor,event_type,impact,urgency,size,event,timing
0,S02,Samsung,pricing_change,9.5,high,10.0,10.0,7.4
1,A01,Apple,product_launch,9.4,immediate,8.4,10.0,10.0
2,X03,Xiaomi,carrier_deal,9.4,high,10.0,10.0,7.0
3,O04,OPPO,camera_award,8.3,high,9.5,8.3,5.9
4,V05,vivo,certification,7.4,high,9.5,6.9,4.4
